In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# Get data from INSEE and from openData

### https://www.data.gouv.fr/fr/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/
### https://www.insee.fr/fr/information/4477356
### this last file has been modified a bit: departement 1->9 were rewritten 01 to 09

In [ ]:
df=pd.read_csv('data/donnees-hospitalieres-classe-age-covid19-2020-04-24-19h00.csv',sep=';')
# use dates: more convenient
df['date']=pd.to_datetime(df['jour'])
print(len(df['reg'].unique()))
print(df.columns)
df[:10]

In [ ]:
dfMvt=pd.read_csv('data/populationMvt/donnees-confinement.csv',sep=';',encoding = "ISO-8859-1")
dfMvt['Scenario1_Nuitees_avant_confinement']=dfMvt['Scenario1_Nuitees_avant_confinement'].str.replace(' ','')
dfMvt['Scenario1_Nuiteestotales_ecart_lie_au_confinement']=dfMvt['Scenario1_Nuiteestotales_ecart_lie_au_confinement'].str.replace(' ','')
habitantFrance=dfMvt[dfMvt['Code_departement']!='FRM']['Scenario1_Nuitees_avant_confinement'].astype(int).sum()

# national count

In [ ]:
df=df.sort_values(by=['reg','date'])
# keep only synthesis class
dfSub=df[df['cl_age90']==0]
#dfGB=pd.DataFrame(dfSub.groupby(['reg','date'])['hosp','rea','rad','dc'].sum()).reset_index()
#print(dfGB[dfGB['date']=='2020-04-12'])
dfGB=pd.DataFrame(dfSub.groupby(['date'])['hosp','rea','rad','dc'].sum()).reset_index()
#print(dfGB[dfGB['date']=='2020-04-12'])

In [ ]:
# to have same yaxis on every chart, take max for of all variables
yaxis_max=np.max(dfGB[['hosp','rea','rad','dc']].max(axis=0))*1.2
yaxis_max=max(1000,yaxis_max)

plt.figure(figsize=(12,8))
plt.subplot(111)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))
plt.plot(dfGB['date'].tolist(),dfGB['hosp'],color='gold',label='nb of people in hospital')
plt.plot(dfGB['date'].tolist(),dfGB['rea'],color='coral',label='nb of people in rea')
plt.plot(dfGB['date'].tolist(),dfGB['rad'],color='green',label='nb of people back home')
plt.plot(dfGB['date'].tolist(),dfGB['dc'],color='crimson',label='nb of people dead')
plt.gcf().autofmt_xdate()
plt.ylim((0,yaxis_max))
plt.title('THIS DAY')
plt.legend(loc='upper left')
plt.show()

In [ ]:
dfSub

# Create basic curves for each department

In [ ]:
dfMvt['Code_departement']

In [ ]:
# use category synthesis to find yaxis maximum
df['sum'] = df[['hosp','rea','dc','rad']].sum(axis=1)
dfSum=df[df['cl_age90']==0]
#print(dfSum)

# remove category synthesis
dfSub=df[df['cl_age90']!=0]
timeStep=4
dateList=np.unique(dfSub['date'].tolist())[::timeStep]
depList=np.unique(dfSub['reg'].tolist())
  
#print(dfSub)

for nbDone,dep in enumerate(depList):
    
    print('start with region',dep,'--------------------------------------------------------------------------------------- ')
    
    depTmp='{:02d}'.format(dep)
    #print('depTmp:',depTmp)
    depNom=dfMvt[dfMvt['Code_departement']==depTmp]['Departement'].tolist()
    print('depNom:',depNom)
        
    if len(depNom)>0:

        yaxis_max=dfSub[dfSub['reg']==dep]['sum'].max(axis=0)*1.1
        #yaxis_max=max(1000,yaxis_max)

        plt.figure(figsize=(6*len(dateList),5))
            
        for num,date in enumerate(dateList):

            data=dfSub[(dfSub['reg']==dep)&(dfSub['date']==date)]

            #print('date:',date)
            #print('data:',data)

            labels=data['cl_age90'].tolist()
            hosp=np.array(data['hosp'].tolist())
            rea=np.array(data['rea'].tolist())
            dc=np.array(data['dc'].tolist())
            rad=np.array(data['rad'].tolist())
            
            #print(len(labels),len(hosp),len(rea),len(dc),len(rad))
            
            plt.subplot(1,len(dateList),num+1)
            width=1.0
            ind=np.arange(len(labels))*2
            #print('ind:',ind)
            plt.bar(ind, hosp, width=width, label='hosp', color='gold',bottom=rad+dc+rea)
            plt.bar(ind,  rea, width=width, label='rea', color='coral',bottom=rad+dc)
            plt.bar(ind,   dc, width=width, label='dc', color='crimson', bottom=rad)
            plt.bar(ind,  rad, width=width, label='rad', color='green')
            plt.ylim((0,yaxis_max))
            plt.xticks(ind,labels)
            plt.ylabel('Number')
            plt.xlabel('age class')
            plt.legend(loc='upper left')
            plt.title('age class distribution on the '+str(pd.to_datetime(date).month)+'/'+str(pd.to_datetime(date).day))
            
        plt.show()
    #if nbDone>5:
    #    break

#points=pd.DataFrame(points,columns=['date','dep','firstDate','habitant','habitantIncrease','admissionIncrease'])

# same but for 10000 inhabitants

In [ ]:
timeStep=4
        
for nbDone,dep in enumerate(depList):
    
    print('start with dep',dep,'--------------------------------------------------------------------------------------- ')
    
    depNom=dfMvt[dfMvt['Code_departement']==dep]['Departement'].tolist()
    
    if len(depNom)>0:
        
        # get the data of population movement for this department
        depNuit=dfMvt[dfMvt['Code_departement']==dep]
        habitant=int(depNuit['Scenario1_Nuitees_avant_confinement'].astype(int))
         
        depNom=depNom[0]
        print('Departement',dep,depNom,'population:',habitant,'part france:{:.2f}%'.format(100*habitant/habitantFrance))
        
        #print(dep)
        depData=dfGB[dfGB['reg']==dep]
        #print(depData)
          
        # compute people getting out of the hospital every day and people dying
        depData['sortie']=depData['rad']-depData['rad'].shift(1)
        depData['dead']=depData['dc']-depData['dc'].shift(1)
        
        # the following addition seems to be wrong ... considering information from the "nouveaux" dataset is safer
        # consequently 'statistics' and acceleration is not computed on these values
        #depData['admission']=depData['hosp']-depData['hosp'].shift(1)+depData['sortie'].shift(1)+depData['dc'].shift(1)        
        #depData['slidingAdmission']=depData['admission'].rolling(3,win_type='triang',center=True).mean()
            
        #print(depData[0:10])

        # to have same yaxis on every chart, take max for of all variables
        yaxis_max=1.1*10000*(np.max(depData[['hosp','rea','rad','dc']].max(axis=0)))/habitant
        yaxis_max=max(10,yaxis_max)
        
        plt.figure(figsize=(12,6))
        plt.subplot(121)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))
        plt.plot(depData['date'].tolist(),10000*depData['hosp']/habitant,color='gold',label='nb of people in hospital')
        plt.plot(depData['date'].tolist(),10000*depData['rea']/habitant,color='coral',label='nb of people in rea')
        plt.plot(depData['date'].tolist(),10000*depData['sortie']/habitant,color='green',label='nb of people back home')
        plt.plot(depData['date'].tolist(),10000*depData['dead']/habitant,color='crimson',label='nb of people dead')
        plt.gcf().autofmt_xdate()
        plt.ylim((0,yaxis_max))
        plt.title('THIS DAY')
        plt.legend(loc='upper left')

        plt.subplot(122)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))
        plt.plot(depData['date'].tolist(),10000*depData['rad']/habitant,color='green',label='nb of people back home')
        plt.plot(depData['date'].tolist(),10000*depData['dc']/habitant,color='crimson',label='nb of people dead')
        plt.gcf().autofmt_xdate()
        plt.ylim((0,yaxis_max))
        plt.title('CUMULATED')
        plt.legend(loc='upper left')
        plt.show()
        
        '''
        plt.subplot(133)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))
        plt.plot(depData['date'].tolist(),depData['sortie'],color='green',label='nb of people back home')
        #plt.plot(depData['date'].tolist(),depData['admission'],color='gold',label='nb of people admitted')
        #plt.plot(depData['date'].tolist(),depData['slidingAdmission'],color='orange',label='nb of people admitted sliding')
        plt.plot(depData['date'].tolist(),depData['dead'],color='crimson',label='nb of people dead')
        plt.gcf().autofmt_xdate()
        plt.ylim((0,yaxis_max))
        plt.title('DAILY')
        plt.legend(loc='upper left')
        plt.show()
        '''
        
        '''
        # get the data of population movement for this department
        depNuit=dfMvt[dfMvt['Code_departement']==dep]
        #print(depNuit)

        habitant=int(depNuit['Scenario1_Nuitees_avant_confinement'].astype(int))
        habitantApres=habitant+int(depNuit['Scenario1_Nuiteestotales_ecart_lie_au_confinement'].astype(int))
        
        habitantIncrease=float((habitantApres/habitant)-1)*100

        #print('habitant:',habitant,habitantApres,habitantIncrease)
            
        dateList=depData[3::timeStep]['date'].tolist()
        
        #print('dateList:',dateList)
                
        firstData=depData[depData['date']==dateList[0]]
        
        print('firstData:',firstData)
        
        for date in dateList:
            thisData=depData[depData['date']==date]
            #print('thisData:',thisData)
            #print(thisData['slidingAdmission'].tolist()[0])
            if int(firstData['slidingAdmission'])>0 and not np.isnan(thisData['slidingAdmission'].tolist()[0]):
                points.append([date,dep,dateList[0],habitant,habitantIncrease,int(thisData['slidingAdmission'])/int(firstData['slidingAdmission'])])
            else:
                points.append([date,dep,dateList[0],habitant,habitantIncrease,np.nan])
       '''     

        #if nbDone>5:
        #    break
            
#points=pd.DataFrame(points,columns=['date','dep','firstDate','habitant','habitantIncrease','admissionIncrease'])

# Sandbox